In [43]:
import os
import cv2
import numpy as np
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import UpSampling2D, AveragePooling2D, Lambda, Conv2D, Conv2DTranspose, Activation, Reshape, concatenate, Concatenate, BatchNormalization, ZeroPadding2D
print('Tensorflow version =', tf.__version__)

Tensorflow version = 1.13.1


In [44]:
def load_image(img_path, show=False):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256,256))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = preprocess_input(img_tensor)

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [45]:
x = []
y = []

def get_image(path):
    data =[]
    for subdir, dirs, files in os.walk(path):
        for f in files:
            path = os.path.join(subdir, f)
            img = load_image(path)       
            data.append(img)
    return data


x = get_image(r'C:\Users\isaac\WowExp\segmentation\ae_data\x')
y = get_image(r'C:\Users\isaac\WowExp\segmentation\ae_data\y')

x = np.asarray(x)
y = np.asarray(y)


In [50]:


def ASPP(tensor):
    '''atrous spatial pyramid pooling'''
    dims = K.int_shape(tensor)
    y_pool = AveragePooling2D(pool_size=(
        dims[1], dims[2]), name='average_pooling')(tensor)
    
    y_pool = Conv2D(filters=128, kernel_size=1, padding='same',
                    kernel_initializer='he_normal', name='pool_1x1conv2d')(y_pool)
    y_pool = BatchNormalization(name=f'bn_1')(y_pool)
    y_pool = Activation('relu', name=f'relu_1')(y_pool)
    
    #y_pool = UpSample.bilinear_upsample(tensor=y_pool, size=[dims[1], dims[2]])
    
    y_pool = UpSampling2D((dims[1], dims[2]))(y_pool)
    y_1 = Conv2D(filters=128, kernel_size=1, dilation_rate=1, padding='same',
                 kernel_initializer='he_normal', name='ASPP_conv2d_d1')(tensor)
    y_1 = BatchNormalization(name=f'bn_2')(y_1)
    y_1 = Activation('relu', name=f'relu_2')(y_1)
    print(y_1.shape)
    y_6 = Conv2D(filters=128, kernel_size=3, dilation_rate=6, padding='same',
                 kernel_initializer='he_normal', name='ASPP_conv2d_d6')(tensor)
    y_6 = BatchNormalization(name=f'bn_3')(y_6)
    y_6 = Activation('relu', name=f'relu_3')(y_6)
    print(y_6.shape)
    y_12 = Conv2D(filters=128, kernel_size=3, dilation_rate=12, padding='same',
                  kernel_initializer='he_normal', name='ASPP_conv2d_d12')(tensor)
    y_12 = BatchNormalization(name=f'bn_4')(y_12)
    y_12 = Activation('relu', name=f'relu_4')(y_12)
    print(y_12.shape)
    y_18 = Conv2D(filters=128, kernel_size=3, dilation_rate=18, padding='same',
                  kernel_initializer='he_normal', name='ASPP_conv2d_d18')(tensor)
    y_18 = BatchNormalization(name=f'bn_5')(y_18)
    y_18 = Activation('relu', name=f'relu_5')(y_18)
    print(y_18.shape)
    y = concatenate([y_pool, y_1, y_6, y_12, y_18], name='ASPP_concat')
    print(y.shape)
    y = Conv2D(filters=128, kernel_size=1, dilation_rate=1, padding='same',
               kernel_initializer='he_normal', name='ASPP_conv2d_final')(y)
    y = BatchNormalization(name=f'bn_final')(y)
    y = Activation('relu', name=f'relu_final')(y)
    return y


def DeepLabV3Plus(img_height, img_width, nclasses=3):
    K.clear_session()
    
    base_model = ResNet50(input_shape=(
        img_height, img_width, 3), weights='imagenet', include_top=False)
    image_features = base_model.get_layer("activation_39").output
    #print(image_features)
    x_a = ASPP(image_features)
    #print(x_a.shape)
    #x_a = UpSample.bilinear_upsample(tensor=x_a, size=[img_height // 4, img_width // 4])
    #x_a = UpSampling2D((img_height, img_width))(x_a)
    x_a = UpSampling2D((4,4))(x_a)
    #print(x_a.shape)
    x_b = base_model.get_layer('activation_9').output
    #print(x_b.shape)
    x_b = Conv2D(filters=48, kernel_size=1, padding='same',
                 kernel_initializer='he_normal', name='low_level_projection')(x_b)
    x_b = BatchNormalization(name=f'bn_low_level_projection')(x_b)
    x_b = Activation('relu', name='low_level_activation')(x_b)

    x = concatenate([x_a, x_b], name='decoder_concat')
    #print(x.shape)
    x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu',
               kernel_initializer='he_normal', name='decoder_conv2d_1')(x)
    x = BatchNormalization(name=f'bn_decoder_1')(x)
    x = Activation('relu', name='activation_decoder_1')(x)

    x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu',
               kernel_initializer='he_normal', name='decoder_conv2d_2')(x)
    x = BatchNormalization(name=f'bn_decoder_2')(x)
    x = Activation('relu', name='activation_decoder_2')(x)
    #x = UpSample.bilinear_upsample(x, [img_height, img_width])
    #print(x.shape)
    x = UpSampling2D((4, 4))(x)
    x = Conv2D(3, (1, 1), name='output_layer')(x)

    model = Model(inputs=base_model.input, outputs=x, name='DeepLabV3_Plus')
    return model
    

In [51]:
model = DeepLabV3Plus(256, 256)
print(model)
model.summary()

(?, 16, 16, 128)
(?, 16, 16, 128)
(?, 16, 16, 128)
(?, 16, 16, 128)
(?, 16, 16, 640)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 128, 128, 64) 256         conv1[0][0]                      
________________________

In [52]:
model = DeepLabV3Plus(256, 256)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

(?, 16, 16, 128)
(?, 16, 16, 128)
(?, 16, 16, 128)
(?, 16, 16, 128)
(?, 16, 16, 640)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 128, 128, 64) 256         conv1[0][0]                      
________________________

In [53]:
#Data augmentation to generate new data from the given data at the time of each batch
# construct the training image generator for data augmentation
batch_size = len(x)
aug = ImageDataGenerator(rotation_range=20)
# train the network
model.fit_generator(aug.flow(x, y, batch_size=batch_size), steps_per_epoch=len(x) // batch_size, epochs=1)

1/1 [==============================] - 52s 52s/step - loss: 2406.1001 - acc: 0.4273


In [ ]:
def plot(img):   
    plt.imshow(img)                           
    plt.axis('off')
    plt.show()

In [ ]:
p = r'C:\Users\isaac\WowExp\segmentation\ae_data\x\1.jpg'
img= cv2.imread(p)
plot(img)

In [ ]:
img = load_image(p)
img = np.expand_dims(img, axis = 0)
result = model.predict(img)
result = cv2.cvtColor(result,cv2.COLOR_GRAY2RGB)
result = np.resize(result, (256,256,3))

In [ ]:
result.shape

In [ ]:
plot(result)